## DEBUG n.1 - Main

In [ ]:
import numpy as np
import re
import matplotlib.pylab as plt 
%matplotlib inline
import os
import tifffile as tiff

In [ ]:
indir = 'H:\\Fanti_Muscioni\\Y3_Xm45\\'

flatdir = os.path.join(indir, 'flat\\')
darkdir = os.path.join(indir, 'dark\\')
tomodir = os.path.join(indir, 'tomo\\')

outdir = os.path.join(indir, 'out\\')

Create output directory

In [ ]:
if not os.path.exists(outdir):
    os.makedirs(outdir)

Rebin function

In [ ]:
def rebin(img, bin_fact):
    """ 
    Scale the image of a n-factor 
    """
    dim1, dim2 = img.shape
    shape = np.array([dim1/bin_fact, dim2/bin_fact]).astype(int)
    sh = shape[0],img.shape[0]//shape[0],shape[1],img.shape[1]//shape[1]
    return img.reshape(sh).mean(-1).mean(1)

Numerical sort function

In [ ]:
def numericalSort(value):
    """ 
    Returns the element in the list or array "value" sorted by their numerical order 
    """
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

Show image function

In [ ]:
def showImage(img, string, dpi):
    dim1, dim2 = img.shape
    fig = plt.figure(frameon = False)
    fig.set_size_inches(dim1/dpi, dim2/dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.imshow(img, cmap = 'gray')
    plt.colorbar()
    plt.title(string)

Find outlier Pixels Function

In [1]:
def find_outlier_pixels(data,worry_about_edges=True):
    #This function finds the hot or dead pixels in a 2D dataset. 
    #tolerance is the number of standard deviations used to cutoff the hot pixels
    #If you want to ignore the edges and greatly speed up the code, then set
    #worry_about_edges to False.
    #
    #The function returns a list of hot pixels and also an image with with hot pixels removed

    from scipy.ndimage import median_filter
    blurred = median_filter(data, size=2)
    difference = data - blurred
    threshold = 10*np.std(difference)

    #find the hot pixels, but ignore the edges
    hot_pixels = np.nonzero((np.abs(difference[1:-1,1:-1])>threshold) )
    hot_pixels = np.array(hot_pixels) + 1 #because we ignored the first row and first column

    fixed_image = np.copy(data) #This is the image with the hot pixels removed
    for y,x in zip(hot_pixels[0],hot_pixels[1]):
        fixed_image[y,x]=blurred[y,x]

    if worry_about_edges == True:
        height,width = np.shape(data)

        ###Now get the pixels on the edges (but not the corners)###

        #left and right sides
        for index in range(1,height-1):
            #left side:
            med  = np.median(data[index-1:index+2,0:2])
            diff = np.abs(data[index,0] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[index],[0]]  ))
                fixed_image[index,0] = med

            #right side:
            med  = np.median(data[index-1:index+2,-2:])
            diff = np.abs(data[index,-1] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[index],[width-1]]  ))
                fixed_image[index,-1] = med

        #Then the top and bottom
        for index in range(1,width-1):
            #bottom:
            med  = np.median(data[0:2,index-1:index+2])
            diff = np.abs(data[0,index] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[0],[index]]  ))
                fixed_image[0,index] = med

            #top:
            med  = np.median(data[-2:,index-1:index+2])
            diff = np.abs(data[-1,index] - med)
            if diff>threshold: 
                hot_pixels = np.hstack(( hot_pixels, [[height-1],[index]]  ))
                fixed_image[-1,index] = med

        ###Then the corners###

        #bottom left
        med  = np.median(data[0:2,0:2])
        diff = np.abs(data[0,0] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[0],[0]]  ))
            fixed_image[0,0] = med

        #bottom right
        med  = np.median(data[0:2,-2:])
        diff = np.abs(data[0,-1] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[0],[width-1]]  ))
            fixed_image[0,-1] = med

        #top left
        med  = np.median(data[-2:,0:2])
        diff = np.abs(data[-1,0] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[height-1],[0]]  ))
            fixed_image[-1,0] = med

        #top right
        med  = np.median(data[-2:,-2:])
        diff = np.abs(data[-1,-1] - med)
        if diff>threshold: 
            hot_pixels = np.hstack(( hot_pixels, [[height-1],[width-1]]  ))
            fixed_image[-1,-1] = med

    return fixed_image,hot_pixels

## Main - Trouble Shooter Flat Field Correction

+ Binning 2 in x and y
+ Binning 2 in z (when required)
+ Problema linee morte detector

In [ ]:
### --- FLAT

flat_list = os.listdir(flatdir)
n_flat = len(flat_list)

flat_n1 = tiff.imread(os.path.join(flatdir, flat_list[0]))
dim1,dim2 = flat_n1.shape

flat_mean = np.zeros([dim1,dim2])
for i in sorted(flat_list, key = numericalSort):
    flat_img = tiff.imread(os.path.join(flatdir,i))
    flat_mean = flat_mean + flat_img 

flat_mean = flat_mean / n_flat
showImage(flat_mean, 'flat average', 400)

### --- DARK

dark_list = os.listdir(darkdir)
n_dark = len(dark_list)

dark_mean = np.zeros([dim1,dim2])
for i in sorted(dark_list, key = numericalSort):
    # dark_img = plt.imread(os.path.join(dark_path,i))
    dark_img = tiff.imread(os.path.join(darkdir,i))
    dark_mean = dark_mean + dark_img

dark_mean = dark_mean / n_dark
showImage(dark_mean,'dark average', 400)

In [ ]:
### --- FLAT - DARK

flat_dark = flat_mean - dark_mean

In [ ]:
### --- Avoid division per 0
div0 = np.argwhere(flat_dark == 0)
print(div0)
for i in range(div0.shape[0]):
    flat_dark[div0[i,0], div0[i,1]] = 1

#showImage(flat_dark, 'flat - dark', 250)

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(18.5, 10.5)
ax1.imshow(flat_dark[550:580,1830:1850])

corrupt_line = 1834

kernel_col = np.column_stack((flat_dark[:,(corrupt_line-1)], flat_dark[:,corrupt_line+2]))
flat_dark = np.column_stack((flat_dark[:,:corrupt_line], np.floor(np.median(kernel_col, axis=1)), flat_dark[:,corrupt_line+1:]))

ax2.imshow(flat_dark[550:580,1830:1850])
#tiff.imsave(os.path.join(outdir, 'flat_dark.tif'), np.rot90(np.uint16(flat_dark)))

In [ ]:
### --- Normalization

v_max = 1.45 # or 1.25 ?
v_min = 0.0

m = 65535 / (v_max - v_min)
q = m*v_min

ALL TOGETHER LOOP:
+ dead pixels correction, dead areas correction, corrupte column correction
+ binning 2 (in z)
+ flat field correction on two images

In [ ]:
corrupt_value = 32768
corrupt_line = 1834

tomo_list = os.listdir(tomodir)
n_tomo = len(tomo_list)

idx = 0
extracted_lines = []
for i in range(0,n_tomo-1,2):
    print(tomo_list[i], '-', tomo_list[i+1])
    img0 = tiff.imread(os.path.join(tomodir, tomo_list[i]))
    img1 = tiff.imread(os.path.join(tomodir, tomo_list[i+1]))
    dead_c0 = np.argwhere(img0 == corrupt_value)
    dead_c1 = np.argwhere(img1 == corrupt_value)
    #print(dead_c1)
    # COLUMNS PROBLEMS
    if dead_c0.shape[0] > 1:
        for j in range(dead_c0.shape[0]):
            img0[dead_c0[j,0], dead_c0[j,1]] = img1[dead_c0[j,0], dead_c0[j,1]] 
    elif dead_c1.shape[0] > 1:
        for j in range(dead_c1.shape[0]):
            img1[dead_c1[j,0], dead_c1[j,1]] = img0[dead_c1[j,0], dead_c1[j,1]]
    # A FEW DEAD PIXELS
    if dead_c0.shape[0] == 1:
        pos0 = dead_c0[0,0]
        pos1 = dead_c0[0,1]
        kernel = np.array([img0[pos0-1:pos0+1, pos1-1], img0[pos0-1:pos0+1,pos1-1]])
        img0[dead_c0[0,0], dead_c0[0,1]] = np.median(kernel) 
    elif dead_c1.shape[0] == 1:
        pos0 = dead_c1[0,0]
        pos1 = dead_c1[0,1]
        kernel = np.array([img1[pos0-1:pos0+1, pos1-1], img1[pos0-1:pos0+1,pos1-1]])
        img1[dead_c1[0,0], dead_c1[0,1]] = np.median(kernel)
    # DEAD PIXELS COLUMN 
    kernel_col = np.column_stack((img0[:,(corrupt_line-1)], img0[:,corrupt_line+2]))
    img0 = np.column_stack((img0[:,:corrupt_line], np.floor(np.median(kernel_col, axis=1)), img0[:,corrupt_line+1:]))
    kernel_col2 = np.column_stack((img1[:,(corrupt_line-1)], img1[:,corrupt_line+2]))
    img1 = np.column_stack((img1[:,:corrupt_line], np.floor(np.median(kernel_col, axis=1)), img1[:,corrupt_line+1:]))
    # MORE DEAD PIXELS
    img0,_ = find_outlier_pixels(img0)
    img1,_ = find_outlier_pixels(img1)
    # FINALLY DO THE FLAT FIELD CORRECTION
    tomo_i0 = np.divide((img0 - dark_mean), (flat_dark))
    tomo_i1 = np.divide((img1 - dark_mean), (flat_dark))
    tomo_img = (tomo_i0 + tomo_i1)/2
    tomo_img = rebin(tomo_img,2)
    tomo_norm = (m*tomo_img)+q
    tomo_def = np.rot90(np.uint16(tomo_norm))
    tiff.imsave(os.path.join(outdir, 'tomo_' + f'{idx:04}' + '.tif'), tomo_def)
    if i == n_tomo-1:
        print('Conversione finita')
        showImage(tomo_def, 'tomo ffc', 250)
        break
    idx += 1

Fight corrupted pixels, corrupted areas and corrupted columns

In [ ]:
corrupt_value = 32768
corrupt_line = 1834

tomo_list = os.listdir(tomodir)
n_tomo = len(tomo_list)

nocorrdir = os.path.join(indir, 'tomo_nocorr\\')
if not os.path.exists(nocorrdir):
    os.makedirs(nocorrdir)
idx = 0
extracted_lines = []
for i in range(0,n_tomo-1,2):
    print(tomo_list[i], '-', tomo_list[i+1])
    img0 = tiff.imread(os.path.join(tomodir, tomo_list[i]))
    img1 = tiff.imread(os.path.join(tomodir, tomo_list[i+1]))
    dead_c0 = np.argwhere(img0 == corrupt_value)
    dead_c1 = np.argwhere(img1 == corrupt_value)
    #print(dead_c1)
    # COLUMNS PROBLEMS
    if dead_c0.shape[0] > 1:
        for j in range(dead_c0.shape[0]):
            img0[dead_c0[j,0], dead_c0[j,1]] = img1[dead_c0[j,0], dead_c0[j,1]] 
    elif dead_c1.shape[0] > 1:
        for j in range(dead_c1.shape[0]):
            img1[dead_c1[j,0], dead_c1[j,1]] = img0[dead_c1[j,0], dead_c1[j,1]]
    # A FEW DEAD PIXELS
    if dead_c0.shape[0] == 1:
        pos0 = dead_c0[0,0]
        pos1 = dead_c0[0,1]
        kernel = np.array([img0[pos0-1:pos0+1, pos1-1], img0[pos0-1:pos0+1,pos1-1]])
        img0[dead_c0[0,0], dead_c0[0,1]] = np.median(kernel) 
    elif dead_c1.shape[0] == 1:
        pos0 = dead_c1[0,0]
        pos1 = dead_c1[0,1]
        kernel = np.array([img1[pos0-1:pos0+1, pos1-1], img1[pos0-1:pos0+1,pos1-1]])
        img1[dead_c1[0,0], dead_c1[0,1]] = np.median(kernel)
    # DEAD PIXELS COLUMN 
    kernel_col = np.column_stack((img0[:,(corrupt_line-1)], img0[:,corrupt_line+2]))
    img0 = np.column_stack((img0[:,:corrupt_line], np.floor(np.median(kernel_col, axis=1)), img0[:,corrupt_line+1:]))
    kernel_col2 = np.column_stack((img1[:,(corrupt_line-1)], img1[:,corrupt_line+2]))
    img1 = np.column_stack((img1[:,:corrupt_line], np.floor(np.median(kernel_col, axis=1)), img1[:,corrupt_line+1:]))
    # Export
    tiff.imsave(os.path.join(nocorrdir, 'tomo_' + f'{i:04}' + '.tif'), img0)
    tiff.imsave(os.path.join(nocorrdir, 'tomo_' + f'{(i+1):04}' + '.tif'), img1)


In [ ]:
### --- TOMO

tomo_list = os.listdir(nocorrdir)
n_tomo = len(tomo_list)

idx = 0
for i in range(0,n_tomo-1,2):
    print(tomo_list[i], '-',tomo_list[i+1])
    tomo_i0 = tiff.imread(os.path.join(nocorrdir,tomo_list[i]))
    tomo_i0 = np.divide((tomo_i0 - dark_mean), (flat_dark))
    #showImage(tomo_i0, 'tomo init', 250)
    tomo_i1 = tiff.imread(os.path.join(nocorrdir,tomo_list[i+1]))
    tomo_i1 = np.divide((tomo_i1 - dark_mean), (flat_dark))
    #showImage(tomo_i0, 'tomo init', 250)
    tomo_img = (tomo_i0 + tomo_i1)/2
    #showImage(tomo_img, 'tomo summed', 250)
    tomo_img = rebin(tomo_img,2)
    tomo_norm = (m*tomo_img)+q
    #showImage(tomo_norm, 'tomo norm', 250)
    tomo_def = np.rot90(np.uint16(tomo_norm))
    #showImage(tomo_def, 'tomo final', 250)
    tiff.imsave(os.path.join(outdir, 'tomo_' + f'{idx:04}' + '.tif'), tomo_def)
    if i == n_tomo-1:
        print('Conversione finita')
        showImage(tomo_def, 'tomo ffc', 250)
        break
    idx += 1

DEBUG SESSION

In [ ]:
corrupt_line = 1834
tomo_list = os.listdir(tomodir)
n_tomo = len(tomo_list)
img0 = tiff.imread(os.path.join(tomodir, tomo_list[i]))
#plt.imshow(img0[550:580,corrupt_line-1:corrupt_line+2]);
kernel = np.column_stack((img0[:,(corrupt_line-1)], img0[:,corrupt_line+2]))
print(kernel.shape)
linea = np.floor(np.median(kernel, axis=1))
print(linea.shape)
# ricompose
fin_img = np.column_stack((img0[:,:corrupt_line], linea, img0[:,corrupt_line+1:]))
fig, (ax1, ax2) = plt.subplots(2)#(1, 2)
fig.set_size_inches(18.5, 10.5)
ax1.imshow(img0[550:580,1830:1850])
print(fin_img.shape)
ax2.imshow(fin_img[550:580,1830:1850])

In [ ]:
### INUTILE
tomo_list = os.listdir(tomodir)
n_tomo = len(tomo_list)

idx = 0
extracted_lines = []
for i in range(0,n_tomo,2):
    img = tiff.imread(os.path.join(tomodir, tomo_list[i]))
    line = img[:, 500]
    extracted_lines.append(line)